In [1]:
import time
import logging
import hashlib
import requests
import sqlite3
from datetime import datetime

# set logging level
logging.basicConfig(level=logging.ERROR)


# Routes to track
# routes = ["701X"]
routes = ["701X", "71", "72", "73", "73S", "N6"]

# remove data from the cache if it is older than this (minutes)
STALE_THRESHOLD_MINUTES = 20
# how long to sleep between posts (seconds)
SLEEP_INTERVAL_SECONDS = 5
# database path
DATABASE_PATH = "./data/bus_data.db"

# Initialize bus station status and last seen timestamps dictionaries
bus_station_status = {}
last_seen_timestamps = {}


In [26]:
# Setup database connection
conn = sqlite3.connect(DATABASE_PATH)
c = conn.cursor()
c.execute(
    """CREATE TABLE IF NOT EXISTS bus_data 
             (route TEXT, bus_plate TEXT, station_code TEXT, arrival_time TEXT, station_index INT)"""
)
conn.commit()


In [25]:
# import sqlite3

# # 连接到数据库
# conn = sqlite3.connect("./data/bus_data.db")
# c = conn.cursor()

# # Step 1: 创建一个临时表，包含每个bus_plate在station_index = 0的两条时间最接近的记录
# c.execute("""
#     CREATE TEMPORARY TABLE TempTable AS
#     SELECT 
#         a.route, 
#         a.bus_plate, 
#         a.station_code, 
#         a.arrival_time, 
#         a.station_index
#     FROM 
#         bus_data a
#     JOIN 
#         bus_data b
#     ON 
#         a.bus_plate = b.bus_plate AND 
#         a.station_index = b.station_index AND 
#         a.arrival_time < b.arrival_time
#     WHERE 
#         a.station_index = 0 AND 
#         b.station_index = 0
#     ORDER BY 
#         a.bus_plate, 
#         a.arrival_time;
# """)

# # Step 2: 从原始表中删除临时表中的较早记录
# c.execute("""
#     DELETE FROM 
#         bus_data
#     WHERE 
#         EXISTS (
#             SELECT 1 
#             FROM 
#                 TempTable
#             WHERE 
#                 bus_data.route = TempTable.route AND 
#                 bus_data.bus_plate = TempTable.bus_plate AND 
#                 bus_data.station_code = TempTable.station_code AND 
#                 bus_data.arrival_time = TempTable.arrival_time AND 
#                 bus_data.station_index = TempTable.station_index
#         );
# """)

# # 提交更改并关闭连接
# conn.commit()
# conn.close()


In [32]:
import pandas as pd
query = """
SELECT arrival_time
FROM bus_data
WHERE route = '701X' AND station_code = 'T415' AND station_index = 6"""
df = pd.read_sql(query, conn)
df.head()

,arrival_time
0,2023-10-05 17:51:20
1,2023-10-05 17:56:34
2,2023-10-05 18:12:09
3,2023-10-05 18:26:54
4,2023-10-05 18:46:25


In [23]:
duplicate_buses.to_csv("./test.csv")

In [13]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM bus_data where route = '701X' AND station_index = 0", conn, parse_dates=["arrival_time"])
df

,route,bus_plate,station_code,arrival_time,station_index
0,701X,AA2459,T550/3,2023-10-05 17:52:02,0
1,701X,AA2459,T550/3,2023-10-05 17:55:40,0
2,701X,AA2259,T550/3,2023-10-05 18:05:50,0
3,701X,AA2259,T550/3,2023-10-05 18:08:03,0
4,701X,AA2314,T550/3,2023-10-05 18:20:12,0
5,701X,AA2314,T550/3,2023-10-05 18:24:21,0
6,701X,AA2930,T550/3,2023-10-05 18:33:23,0
7,701X,AA2930,T550/3,2023-10-05 18:33:45,0
8,701X,AA3018,T550/3,2023-10-05 18:46:58,0
9,701X,AA3018,T550/3,2023-10-05 18:48:23,0


In [11]:
df["arrival_time"][0]

Timestamp('2023-10-05 17:48:54')

In [2]:

# Setup post request data
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.47"
}
# bus post url
bus_request_url = "https://bis.dsat.gov.mo:37812/macauweb/routestation/bus"
# post data
fix_payload = {
    "action": "dy",
    # direction: 0 or 1; always 0 for loop line
    "dir": "0",
    "lang": "zh-tw",
    "device": "web",
}


# build token in header
def get_token(route, payload=fix_payload):
    """Construct token for bus request.
    Parameters
    ----------
    route : str
        Route id like 701X, 71, 72, 73, 73S, N6
    payload : dict
        Fixed payload for bus request

    Returns
    -------
    payload : dict
        Updated payload with bus id
    token : str
        Token for bus request
    """

    # get md5 hash
    def get_bus_md5(payload):
        # suffix like "action=dy&routeName=73&dir=0&lang=zh-tw&device=web"
        bus_url_suffix = "&".join([f"{k}={v}" for k, v in payload.items()])
        return hashlib.md5(bus_url_suffix.encode("utf-8")).hexdigest()

    # add bus id to payload
    payload["routeName"] = route
    bus_md5 = get_bus_md5(payload)
    # get current time and format in "YYYYMMDDHHmm"
    current_time = datetime.now().strftime("%Y%m%d%H%M")
    # concatenate token and current time
    return (
        payload,
        bus_md5[:4]
        + current_time[:4]
        + bus_md5[4:12]
        + current_time[4:8]
        + bus_md5[12:24]
        + current_time[8:]
        + bus_md5[24:],
    )


def record_bus_arrival(route, bus_plate, station_code, station_index):
    """Record bus arrival in database.

    Parameters
    ----------
    route : str
        Route id like 701X, 71, 72, 73, 73S, N6
    bus_plate : str
        Bus plate number
    station_code : str
        Station code like T530/1
    station_index : int
        Station index in the route
    """
    arrival_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute(
        "INSERT INTO bus_data VALUES (?,?,?,?,?)",
        (route, bus_plate, station_code, arrival_time, station_index),
    )
    conn.commit()
    logging.info(
        f"{route}: Bus {bus_plate} arrived at station {station_code} ({station_index}) at {arrival_time}"
    )


def get_api_response(route):
    """Post request to get bus info.
    Parameters
    ----------
    route : str
        Route id like 701X, 71, 72, 73, 73S, N6
    """
    with requests.Session() as session:
        payload, headers["token"] = get_token(route)
        response = session.post(bus_request_url, headers=headers, data=payload)
        response.raise_for_status()
        return response


def process_response(route, response_json):
    """Process response from bus request.
    Parameters
    ----------
    route : str
        Route id like 701X, 71, 72, 73, 73S, N6
    response_json : dict
        Response from bus request
    """
    # Get route info
    route_info = response_json.get("data", {}).get("routeInfo", [])
    if not route_info:
        logging.error("routeInfo is empty")
        return
    last_station_index = len(route_info) - 1
    # Get current timestamp
    current_timestamp = datetime.now().timestamp()
    # Get bus info in each station
    for station_index, station in enumerate(route_info):
        sta_code = station["staCode"]
        for bus in station.get("busInfo", []):
            bus_plate = bus["busPlate"]
            status = bus["status"]
            # Get the previous status and pass count of the bus
            previous_status = bus_station_status.get(bus_plate, {}).get(sta_code, None)
            # If the bus was not in the station last time but is now, record arrival and increment pass count
            if previous_status != "1" and status == "1":
                record_bus_arrival(route, bus_plate, sta_code, station_index)
            # Update the bus station status and last seen timestamp
            bus_station_status.setdefault(bus_plate, {})[sta_code] = status
            last_seen_timestamps[bus_plate] = current_timestamp
            # If the bus is at the last station, remove its status
            if station_index == last_station_index:
                bus_station_status.pop(bus_plate, None)
                last_seen_timestamps.pop(bus_plate, None)

    # Remove data for buses not seen for STALE_THRESHOLD_MINUTES minutes
    stale_timestamp = current_timestamp - STALE_THRESHOLD_MINUTES * 60
    stale_buses = [
        bus_plate
        for bus_plate, timestamp in last_seen_timestamps.items()
        if timestamp < stale_timestamp
    ]
    for bus_plate in stale_buses:
        bus_station_status.pop(bus_plate, None)
        last_seen_timestamps.pop(bus_plate, None)

In [8]:
get_api_response("701X").json()

{'data': {'busColor': 'Orange',
  'lastBusPlate': '',
  'routeInfo': [{'staCode': 'T550/3', 'busInfo': []},
   {'staCode': 'T560/5', 'busInfo': []},
   {'staCode': 'T372', 'busInfo': []},
   {'staCode': 'T360', 'busInfo': []},
   {'staCode': 'T376/1',
    'busInfo': [{'busPlate': 'AA3018',
      'busCode': 'E3257',
      'status': '0',
      'isFacilities': '0',
      'passengerFlow': '-1',
      'speed': '29',
      'busType': '1'}]},
   {'staCode': 'T414', 'busInfo': []},
   {'staCode': 'T415', 'busInfo': []},
   {'staCode': 'T384', 'busInfo': []},
   {'staCode': 'T436', 'busInfo': []},
   {'staCode': 'T434', 'busInfo': []},
   {'staCode': 'T400', 'busInfo': []},
   {'staCode': 'T392', 'busInfo': []},
   {'staCode': 'T367', 'busInfo': []},
   {'staCode': 'T394', 'busInfo': []},
   {'staCode': 'T396', 'busInfo': []},
   {'staCode': 'T377', 'busInfo': []},
   {'staCode': 'T359', 'busInfo': []},
   {'staCode': 'T355/2', 'busInfo': []},
   {'staCode': 'T560/7',
    'busInfo': [{'busPlate

In [ ]:
import sqlite3

conn = sqlite3.connect(DATABASE_PATH)
